# Imports

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd

/home/mp/Desktop/grocery_swiper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load models

In [2]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-da-en", device='cpu')

model_name = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cpu"
)

/home/mp/Desktop/grocery_swiper/.venv/lib/python3.13/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 47.88it/s]


# Translate product to english, then generate bio

In [3]:
def product_to_bio(product_name):
    # translate product name from Danish to English
    translation = translator(product_name, max_length=40)[0]['translation_text']
    
    # prepare the model input
    prompt = f"Give me a fun and creative short single-sentence Tinder bio for me, but imagine I am the following item from the grocery store: '{translation}'. Only output the bio."
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Disable thinking mode
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # Disable thinking mode here
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=100
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # decode final output normally
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")

    return content, translation

# Generate bios for multiple products

In [4]:
df_products = pd.read_csv("../data/csv/products_2025-10-18.csv")
df_products

,data_id,price,brand,category,product_name,units,quantity,unit_type,store_name,image_url,start_date,end_date
0,10764946,8.0,Hello Sensitive,Babypleje,Vådservietter,1,100,pk.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
1,10764948,10.0,Hello Sensitive,Babypleje,Skumklude,1,50,pk.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
2,10764968,89.0,Fitness Pharma,Kosttilskud,Mineraler,1,250,bg.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
3,10764969,89.0,Fitness Pharma,Kosttilskud,Kosttilskud,1,200,ds.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
4,10764967,89.0,Fitness Pharma,Fiskeolie,Fiskeolie,1,200,ds.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...
388,10764691,40.0,NaN,Cykeltilbehør,Cykelovertræk,1,0,stk.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
389,10764694,40.0,NaN,Cykeltilbehør,Cykellås,1,0,stk.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
390,10764696,30.0,NaN,Cykeltilbehør,Cykelkurvovertræk,1,0,stk.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24
391,10764709,119.0,NaN,Boligtekstiler,Duge,1,0,stk.,Netto,https://static.tilbudsugen.dk/1st-retail/2025/...,2025-10-18,2025-10-24


In [5]:
for product in df_products['product_name'].iloc[:5]:
    content, translation = product_to_bio(product)
    print(f"Product: {product}")
    print(f"Translation: {translation}")
    print(f"Tinder Bio: {content}")
    print("-----")

Product: Vådservietter
Translation: Wet napkins
Tinder Bio: I’m not just a wipe—I’m a splash of freshness in your life, one absorbent moment at a time. 🧺✨
-----
Product: Skumklude
Translation: Foam cloths
Tinder Bio: I’m the silent hero of every scrub, the unsung hero of every wipe, and the secret sauce of every clean surface—because even the most mundane things can be *extra* magical. 🧴✨
-----
Product: Mineraler
Translation: Minerals
Tinder Bio: I’m the secret ingredient in every healthy recipe—no flavor, just pure nutrition, and a little bit of magic.
-----
Product: Kosttilskud
Translation: Food supplements
Tinder Bio: I’m the grocery store’s secret superpower—boosting your health one pill at a time.
-----
Product: Fiskeolie
Translation: Fish oil
Tinder Bio: I’m not just a supplement—I’m the superfood that gives your body a fishy twist on wellness. 🐟✨
-----
